# Plot comparison of costs of restricted and open designs, and resulting VoI estimate convergence

In [ ]:
# Hack to emulate running notebook from root directory.
import os
os.chdir('..')

In [ ]:
import yaml
import numpy as np

from configs import get_experiment_config

from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors

#### Load design results for selected experiment

In [ ]:
expt_id = 1
settings, base_params = get_experiment_config(expt_id)
available_technologies = list(settings['probability_settings']['storage'].keys())
N = settings['probability_settings']['n_prior_samples']

In [ ]:
result_dir_path_pattern = os.path.join(*settings['results_dir'],'posterior','z_scenario_{z}')

open_design_results_files = [os.path.join(result_dir_path_pattern.format(z=z), 'open_design.yaml') for z in range(N)]
open_results = [yaml.safe_load(open(f)) for f in open_design_results_files]
restricted_design_results_files = [os.path.join(result_dir_path_pattern.format(z=z), 'restricted_design.yaml') for z in range(N)]
restricted_results = [yaml.safe_load(open(f)) for f in restricted_design_results_files]

Quick bit of pre-analysis

In [ ]:
# Compute VoI
open_expected_cost = np.mean([res['overall_objective']['overall_objective'] for res in open_results])
restricted_expected_cost = np.mean([res['overall_objective']['overall_objective'] for res in restricted_results])
print(open_expected_cost/1e6)
print(restricted_expected_cost/1e6)

voi = restricted_expected_cost - open_expected_cost
print(voi/1e6)
print(voi/np.abs(restricted_expected_cost)*100)

In [ ]:
# Look at frequency of posterior optimal storage technologies
techs = [res['design']['storage_technologies'][0] for res in open_results]
counts = Counter(techs)
print(dict(counts))

#### Plot designs

In [ ]:
colors = list(mcolors.TABLEAU_COLORS.values())

In [ ]:
fig, ax = plt.subplots()

restricted_tech = restricted_results[0]['design']['storage_technologies'][0]
restricted_wind_caps = np.array([])
restricted_storage_caps = np.array([])

for res in restricted_results:
    restricted_wind_caps = np.append(restricted_wind_caps,res['design']['wind_capacity']['value'])
    restricted_storage_caps = np.append(restricted_storage_caps,res['design']['storage_capacities'][restricted_tech]['value'])

open_wind_caps = np.array([])
open_techs = np.array([])
open_storage_caps = np.array([])

for res in open_results:
    open_wind_caps = np.append(open_wind_caps,res['design']['wind_capacity']['value'])
    tech = res['design']['storage_technologies'][0]
    open_techs = np.append(open_techs,tech)
    open_storage_caps = np.append(open_storage_caps,res['design']['storage_capacities'][tech]['value'])

ax.scatter(
    x=restricted_wind_caps[open_techs != restricted_tech]/1e6,
    y=restricted_storage_caps[open_techs != restricted_tech]/1e6,
    c=colors[available_technologies.index(restricted_tech)],
    alpha=0.5,
    lw=0,
    label='Restricted',
    marker='s'
)

for tech in available_technologies:
    ax.scatter(
        x=open_wind_caps[open_techs == tech]/1e6,
        y=open_storage_caps[open_techs == tech]/1e6,
        c=colors[available_technologies.index(tech)],
        label=tech
    )

plt.xlabel('Wind Capacity (GW)')
plt.ylabel('Storage Capacity (GWh)')

plt.legend()
plt.show()

#### Plot improvement in cost from open design c.f. restricted

In [ ]:
open_costs = np.array([])
restricted_costs = np.array([])
open_techs = np.array([])

for i in range(N):
    open_costs = np.append(open_costs,open_results[i]['overall_objective']['overall_objective'])
    restricted_costs = np.append(restricted_costs,restricted_results[i]['overall_objective']['overall_objective'])
    open_techs = np.append(open_techs,open_results[i]['design']['storage_technologies'][0])

cost_improvements = restricted_costs - open_costs

voi_estimates = np.array([np.mean(cost_improvements[:i+1]) for i in range(N)])
voi_std_errors = np.array([np.std(cost_improvements[:i+1])/np.sqrt(i+1) for i in range(N)])

In [ ]:
# plot cost comparison open vs restricted case
fig, ax = plt.subplots()

sns.histplot(
    cost_improvements/1e6,
    binrange=(0,100),
    binwidth=5,
    ax=ax,
    color='black'
)

plt.xlim(0,100)

plt.xlabel('Cost Improvement (EUR/yr, millions)')
plt.ylabel('No. of Scenarios')

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(
    np.arange(1,N+1),
    voi_estimates/1e6,
    label='VoI estimate',
    color='black',
    lw=2.5,
    zorder=1
)
ax.fill_between(
    np.arange(1,N+1),
    (voi_estimates - 2*voi_std_errors)/1e6,
    (voi_estimates + 2*voi_std_errors)/1e6,
    label='95% CI',
    color='black',
    lw=0,
    alpha=0.2,
    zorder=0
)

for tech in available_technologies:
    ax.scatter(
        x=np.arange(1,N+1)[open_techs == tech],
        y=cost_improvements[open_techs == tech]/1e6,
        label=tech,
        clip_on=False,
        zorder=100
    )

plt.xlim(0,N+1)
plt.ylim(0)

plt.xlabel('Scenario')
plt.ylabel('Cost Improvement (EUR/yr, millions)')

plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.plot(
    np.arange(1,N+1),
    voi_estimates/1e6,
    color='black',
    lw=2.5,
    zorder=1
)
ax.fill_between(
    np.arange(1,N+1),
    (voi_estimates - 2*voi_std_errors)/1e6,
    (voi_estimates + 2*voi_std_errors)/1e6,
    color='black',
    lw=0,
    alpha=0.2,
    zorder=0
)

plt.xlim(0,N)
plt.ylim(0)

plt.xlabel('Scenario')
plt.ylabel('VoI estimate (EUR/yr, millions)')

plt.show()

Plot scenarios ordered by cost improvement

In [ ]:
ordered_cost_improvements = cost_improvements[np.argsort(cost_improvements)]
ordered_open_techs = open_techs[np.argsort(cost_improvements)]


fig, ax = plt.subplots()

ax.plot(
    np.arange(0,N+2),
    np.ones(N+2)*np.mean(ordered_cost_improvements)/1e6,
    label='VoI',
    color='black',
    lw=2.5,
    zorder=1
)
for tech in available_technologies:
    ax.scatter(
        x=np.arange(1,N+1)[ordered_open_techs == tech],
        y=ordered_cost_improvements[ordered_open_techs == tech]/1e6,
        label=tech,
        clip_on=False
    )

plt.xlim(0,N+1)
plt.ylim(0)

plt.xlabel('Ordered Scenarios')
plt.ylabel('Cost Improvement (EUR, millions)')

plt.legend()
plt.show()